# Деление на слоги

In [2]:
from rusyll import rusyll

Пример работы библиотеки:

In [3]:
rusyll.token_to_syllables("черепушидзе")

['че', 'ре', 'пу', 'ши', 'дзе']

# Ударения

In [5]:
from russtress import Accent

Пример работы библиотеки:

In [6]:
accent = Accent()
text = 'Я вернулся в мой город, знакомый до слёз,'
accented_text = accent.put_stress(text)
accented_text

"Я верну'лся в мой го'род, знако'мый до слёз,"

# Ударение + Деление на слоги

In [7]:
test = 'Я вернулся в мой город, знакомый до слёз,'

In [8]:
li_unstressed = ['до', 'за', 'без', 'из', 'по']

In [9]:
vowels = 'аеёиоуыэюя'

Ф-я добавляет графическое ударение там, где библиотечная функция не ставит (а именно в словах с одним гласным и на ё)

In [10]:
def add_stress(text):
  res = []
  words = text.split()
  for word in words:
    w = word.strip(' .,-!?()\\"\\/*#№')
    if w not in li_unstressed and '\'' not in w:
      #print(True)
      letters = []
      for i in word:
        letters.append(i)
        if i.lower() in vowels:
          letters.append('\'')
      stress_fixed = ''.join(letters)
      res.append(stress_fixed)
    else:
      res.append(word)
  return " ".join(res)

In [11]:
add_stress(accented_text)

"Я' верну'лся в мо'й го'род, знако'мый до слё'з,"

Ф-я совмещает деление на слоги и графическое ударение

In [12]:
def rhythm(text):
  syl = '|'.join(rusyll.token_to_syllables(text))
  accented_previous = accent.put_stress(text)
  accented = add_stress(accented_previous)


  new_line = []
  max_len = max(len(text), len(syl), len(accented))
  idx_syl = 0
  idx_accented = 0
  idx_text = 0
  while idx_accented < max_len and idx_syl < max_len:
    #print(idx_accented, idx_syl, max_len)
    if text[idx_text] == accented[idx_accented]:
      idx_accented += 1
    else:
      new_line += accented[idx_accented]
      idx_accented += 2
    if text[idx_text] == syl[idx_syl]:
      idx_syl += 1
    else:
      new_line += syl[idx_syl]
      idx_syl += 2

    new_line += text[idx_text]
    idx_text += 1

  res = "".join(new_line)
  return res

In [13]:
'|'.join(rusyll.token_to_syllables('Тот, кто любит цветы,'))

'То|т, кто| лю|би|т цве|ты,'

In [14]:
rhythm(test)

"Я'| вер|ну'л|ся| в мо'й| го'|ро|д, зна|ко'|мый| до| слё'з,"

Ф-я превращает результат в список нулей и единиц, где 1 - ударный слог, 0 - безударный слог

In [15]:
def rhythm_to_pattern(rhythmed):
  pattern = []
  data = rhythmed.split('|')
  for i in data:
    if '\'' in i:
      pattern.append(1)
    else:
      pattern.append(0)
  return pattern

In [16]:
rhythm_to_pattern(rhythm(test))

[1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]

## Корпус

In [18]:
from google.colab import files
files.upload()

In [19]:
import csv

In [20]:
corpus = []
with open('poems_clean1.csv', encoding='utf-8', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter='|')
    for row in reader:
        corpus.append(row)

In [21]:
corpus_clean = []
for i in corpus:
  if i:
    corpus_clean.append(i)

In [22]:
def assess_rhythm(poem):
  lines = poem.split("\r\n")
  true_lines = []
  for i in lines:
    if i:
      true_lines.append(i)

  li_rhythm = []
  for line in true_lines:
    li_rhythm.append(rhythm_to_pattern(rhythm(line)))

  return li_rhythm

Sample1

In [25]:
sample1 = corpus_clean[-3][3]
sample1

'\r\nВ дали мирозданной тайны,\r\nГде вечность розой цветёт,\r\nРасцветает моя страсть в слове,\r\nИ поэзия в сердце горит.\r\nВ ночной глубине мечты,\r\nГде звёзды сияют над нами,\r\nЯ раскрываю свои крылья,\r\nИ слова мои—как дым ветрами.\r\nОни несутся, легки и воздушны,\r\nКак звуки чарующей песни,\r\nОни погружают меня в иные миры,\r\nГде с ними свободно мне быть одной.\r\nСтихи, как реки из настроений,\r\nЛьются из души, страсти огненной,\r\nОни рисуют картинами звучными,\r\nМинуты вечности соединяющими.\r\nЯ словами вмещаю мир,\r\nПлету романы и сказки,\r\nЯ в каждой строчке пророчу,\r\nТепло и свет, надежду и радость.\r\nСтихами снаряжаюсь в путь,\r\nГде смысл ищу и создаю,\r\nЯ путешествую по красотам,\r\nЧто навсегда в сердце останутся.\r\nИ когда на странице белой\r\nВсе строчки расположатся в техе,\r\nТо для меня весь мир оживает,\r\nВ словах живёт и дышит светом.'

In [24]:
assess_rhythm(sample1)

[[1, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 1],
 [1, 1, 0, 0, 1, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 1, 0, 0, 1],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 0]]

Sample2

In [26]:
sample2 = corpus_clean[1070][3]
sample2

'\r\nБыстро проходят минуты,\r\nА с ними часы,\r\nИ те яркие дни,\r\nЧто были мы с тобой уединены.\r\n\r\nЯ чуствую, что теряю тебя,\r\nНо при этом понимаю себя.\r\nЗаметно уносится время,\r\nИ не попробуем больше мы этого крема,\r\n\r\nСделанного из наших прикосновений,\r\nИ тех счастливых мгновений,\r\nКогда с тобой были мы,\r\nВысокими чувствами переполнены.\r\n\r\nТеперь ты свободна,\r\nТеперь ты одна,\r\nНо знай, я буду любить тебя всегда,\r\nИ надеюсь, не забудешь меня никогда!\r\n\r\nХотя о чем я мечтаю,\r\nВсе закончилось в ту ночь,\r\nКогда нас одолела горечь,\r\nИ теперь живя я умираю...\r\n\r\n\r\n\r\n'

In [27]:
assess_rhythm(sample2)

[[1, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 0, 1],
 [1, 1, 1, 0, 0, 1],
 [1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
 [1, 1, 0, 0, 1, 0, 1, 0, 0, 1],
 [1, 1, 1, 0, 0, 0, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 1, 0],
 [0, 1, 0, 1, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 1, 1, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 1, 0, 0, 1],
 [1, 0, 1, 0, 1, 1, 0, 0, 1, 0]]

Sample3

In [28]:
sample3 = corpus_clean[1003][3]
sample3

'\r\nКто узнал в этих шутках себя,\r\nИзвини! Я не думал обидеть\r\nНикого из девчат и ребят.\r\nИ не знал, что вот так оно выйдет.\r\n\r\nЯ шутил просто так, ни о ком,\r\nНапридумывал всех персонажей.\r\nДаже не было мысли о том,\r\nЧто всё это не чистая лажа.\r\n\r\nТак что если обиделся ты\r\nВ этих шутках себя узнавая,\r\nОщущаешь злость, ярость и стыд,\r\nИзвини! Я тебя умоляю.\r\n\r\n\r\n\r\n'

In [29]:
assess_rhythm(sample3)

[[1, 0, 1, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
 [1, 1, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 1, 0, 1, 1],
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0]]

Sample4

In [30]:
sample4 = corpus_clean[3][3]
sample4

'\r\nК Керн*\r\nЯ помню чудное мгновенье:\r\nПередо мной явилась ты,\r\nКак мимолетное виденье,\r\nКак гений чистой красоты.\r\nВ томленьях грусти безнадежной,\r\nВ тревогах шумной суеты,\r\nЗвучал мне долго голос нежный\r\nИ снились милые черты.\r\nШли годы. Бурь порыв мятежный\r\nРассеял прежние мечты,\r\nИ я забыл твой голос нежный,\r\nТвои небесные черты.\r\nВ глуши, во мраке заточенья\r\nТянулись тихо дни мои\r\nБез божества, без вдохновенья,\r\nБез слез, без жизни, без любви.\r\nДуше настало пробужденье:\r\nИ вот опять явилась ты,\r\nКак мимолетное виденье,\r\nКак гений чистой красоты.\r\nИ сердце бьется в упоенье,\r\nИ для него воскресли вновь\r\nИ божество, и вдохновенье,\r\nИ жизнь, и слезы, и любовь.\r\n\r\n'

In [31]:
assess_rhythm(sample4)

[[1],
 [1, 1, 0, 0, 1, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 1],
 [1, 1, 0, 1, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, 0, 1],
 [1, 1, 0, 1, 1, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 1],
 [1, 1, 0, 1, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 1, 0, 1],
 [1, 0, 0, 1, 1, 0, 0, 1, 0],
 [1, 1, 1, 0, 1, 1, 0, 1]]

**Ритм стихотворения характеризуется метром и размером.**

**Метр** - это повторение определенного соотношения ударного и безударного слогов. Например, если мы обозначим единицей ударные слоги, а нулем - безударные, то ямб можно записать как 01010101, дактиль - 100100100100.
**Размер** - это количество стоп (стопа - повторяющаяся группа ударных и безударных слогов в строке), то есть, проще говоря, размер - это длина строки.

Мы видим на сэмплах, что метр не определяется правильно нашим кодом (поскольку нет закономерности для каждой строки). Скорее всего это связано с тем, что в стихотворениях ударными могут являться не только те слоги, которые ударны "согласно словарю". Напротив, размер легко определить с помощью простого подсчета слогов в каждой строке. Это нормально, что длины чередуются (как в сэмплах 3 и 4, например) или даже имеют сложную схему длин, но стихотворение нельзя признать ритмичным, если длины строк произвольны (как в сэмплах 1 и 2). Поэтому в качестве показателя ритма мы возьмем отношение количества разных длин строк в стихе к его длине (количеству строк). Так, у сэмпла №3 всего две длины строк и всего двенадцать строк, поэтому показатель ритма будет равен 2/12 = 0.17. Поскольку при таком подходе мы получаем ситуацию, когда чем показатель меньше, тем лучше (в то время как другие параметры разметки ориентированы на противоположную ситуацию - "чем больше, тем лучше"), мы вычтем полученные показатели ритма из 1, чтобы инвертировать значения.

In [ ]:
def assess_syl_num(poem):
  lines = poem.split("\r\n")
  true_lines = []
  for i in lines:
    if i:
      true_lines.append(i)

  li_syl = []
  for line in true_lines:
    li_syl.append(len(rusyll.token_to_syllables(line)))

  len_diff_len_lines = 1 - len(list(set(li_syl))) / len(true_lines)
  len_diff = len(list(set(li_syl)))
  len_lines = len(true_lines)

  return [len_diff_len_lines, len_diff, len_lines]

In [ ]:
corpus_syl = []
corpus_syl.append(corpus_clean[0])
for i in corpus_clean[1:]:
  res = i.copy()
  syl_res = assess_syl_num(i[1])
  res.append(syl_res[0])
  res.append(syl_res[1])
  res.append(syl_res[2])
  corpus_syl.append(res)

In [ ]:
with open('syl_poems1.csv', 'w', encoding='utf-8') as f_w:
  csvwriter = csv.writer(f_w)
  csvwriter.writerow(['', 'text', 'name', 'difference in lines (divided by len)', 'difference in lines', 'len'])
  csvwriter.writerows(corpus_syl)